# Run macro file 

This notebook shows how load an ImageJ macrofile linked to a dataset and analyze all the images contained in that dataset with the macro.
This notebook assumes that a macro file has been linked to the dataset. If more than one macro is linked, the first one found will be applied.

Fiji has been installed with few other plugins. For more information, check the Dockerfile.

In [1]:
//Add dependencies to the classpath
%classpath add jar /opt/java-apps/Fiji.app/jars/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/OMERO.imagej-5.5.6/*

### Description

The following section shows
 * how to load a macro file linked to a dataset
 * how to analyze all the images in the dataset using the macro
 

In [2]:
g = new EasyForm("Enter credentials and continue to the next cell. Do not re-run this cell")
g.addTextField("Server").onInit({g['Server'] = "outreach.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g


### Connect to OMERO

In [3]:
import omero.gateway.Gateway
import omero.gateway.LoginCredentials
import omero.log.SimpleLogger

// Method to connect to OMERO
def connect_to_omero() {
    "Connect to OMERO"

    credentials = new LoginCredentials()
    credentials.getServer().setHostname(g['Server'])
    credentials.getServer().setPort(4064)
    credentials.getUser().setUsername(g['UserName'].trim())
    credentials.getUser().setPassword(g['Password'].trim())
    simpleLogger = new SimpleLogger()
    gateway = new Gateway(simpleLogger)
    gateway.connect(credentials)
    return gateway

}

// Connect to OMERO
println "connecting..."
gateway = connect_to_omero()
println "connected..."


connecting...
connected...


In [4]:
ga = new EasyForm("Select the Dataset to analyze. Do not re-run this cell")
ga.addTextField("DatasetID")
ga

### Run the macro

In [ ]:
import java.util.ArrayList
import java.lang.Math
import java.lang.StringBuffer
import java.nio.ByteBuffer
import java.io.File
import java.io.PrintWriter

import java.nio.file.Files

import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility
import omero.gateway.facility.MetadataFacility
import omero.gateway.facility.ROIFacility
import omero.gateway.facility.TablesFacility
import omero.gateway.model.FileAnnotationData
import omero.gateway.model.MapAnnotationData
import omero.gateway.model.DatasetData
import omero.gateway.model.ImageData
import omero.gateway.model.ProjectData
import omero.gateway.model.TableData
import omero.gateway.model.TableDataColumn


import omero.model.DatasetI
import omero.model.ImageI
import omero.model.ProjectI
import omero.log.SimpleLogger
import omero.model.ChecksumAlgorithmI
import omero.model.FileAnnotationI
import omero.model.OriginalFile
import omero.model.OriginalFileI
import omero.model.enums.ChecksumAlgorithmSHA1160

import static omero.rtypes.rlong
import static omero.rtypes.rstring


import org.openmicroscopy.shoola.util.roi.io.ROIReader

import ij.IJ
import ij.plugin.frame.RoiManager
import ij.measure.ResultsTable

//Connection information
HOST = g['Server']
USERNAME = g['UserName']
PASSWORD = g['Password']

PORT = 4064
//size of the chunk to load
INC = 262144
//Dataset to analyze Convert the String as long
dataset_id = ga['DatasetID'].toLong()


// Helper methods


def get_images(gateway, dataset_id) {
    "List all image's ids contained in a Dataset"
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    browse = gateway.getFacility(BrowseFacility)
    ids = new ArrayList(1)
    ids.add(new Long(dataset_id))
    return browse.getImagesForDatasets(ctx, ids)
}

def open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(g['Server'])
    options.append("\nuser=")
    options.append(g['UserName'].trim())
    options.append("\nport=")
    options.append(4064)
    options.append("\npass=")
    options.append(g['Password'].trim())
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

// Load the macro file
def get_macro_file(gateway, dataset_id) {
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    svc = gateway.getFacility(MetadataFacility)
    //Find the dataset
    browse = gateway.getFacility(BrowseFacility)
    object = browse.findIObject(ctx, "omero.model.Dataset", dataset_id)
    dataset = new DatasetData(object);
    users = new ArrayList(1)
    users.add(new Long(user.getId()))
    types = new ArrayList(1)
    types.add(FileAnnotationData.class)
    annotations = svc.getAnnotations(ctx, dataset, types, users)
    
    store = gateway.getRawFileService(ctx);
    querySvc = gateway.getQueryService(ctx)
    index = 0
    annotations.each { fa ->
        of = (OriginalFile) querySvc.get("omero.model.OriginalFile", fa.getFileID());
        name = of.getName().getValue()
        //load the first annotation with ijm
        if (name.endsWith(".ijm") && index == 0) {
            file = File.createTempFile(name, ".ijm")
            stream = new FileOutputStream(file)
            store.setFileId(fa.getFileID());
            offset = 0;
            size = of.getSize().getValue();
            try {
                for (offset = 0; (offset+INC) < size;) {
                    stream.write(store.read(offset, INC));
                    offset += INC;
                }
            } finally {
                stream.write(store.read(offset, (int) (size-offset)));
                stream.close();
            }
            index++
        }
    }
    store.close()
    return file
}

// save the ROI
def save_rois_to_omero(ctx, image_id, imp) {
    // Save ROI's back to OMERO
    reader = new ROIReader()
    roi_list = reader.readImageJROIFromSources(image_id, imp)
    roi_facility = gateway.getFacility(ROIFacility)
    result = roi_facility.saveROIs(ctx, image_id, exp_id, roi_list)

    roivec = new ArrayList()

    j = result.iterator()
    while (j.hasNext()) {
        roidata = j.next()
        roi_id = roidata.getId()

        i = roidata.getIterator()
        while (i.hasNext()) {
            roi = i.next()
            shape = roi[0]
            t = shape.getZ()
            z = shape.getT()
            c = shape.getC()
            shape_id = shape.getId()
            roivec.add([roi_id, shape_id, z, c, t])
        }
    }
    return roivec
}

// Prepare and save the results table
def save_row(rt, table_rows, channel_index, dataset_name, image) {
    "Create a summary table from the original table"
    // Remove the rows not corresponding to the specified channel
    to_delete = new ArrayList()
    
    ref = "c:" + channel_index
    max_bounding_box = 0.0f
    for (i = 0; i < rt.size(); i++) {
        label = rt.getStringValue("Label", i)
        if (label.contains(ref)) {
            w = rt.getStringValue("Width", i)
            h = rt.getStringValue("Height", i)
            area = Float.parseFloat(w) * Float.parseFloat(h)
            max_bounding_box = Math.max(area, max_bounding_box)
        }
    }
    // Rename the table so we can read the summary table
    IJ.renameResults("Results")
    rt = ResultsTable.getResultsTable()
    for (i = 0; i < rt.size(); i++) {
        value = rt.getStringValue("Slice", i)
        if (!value.startsWith(ref)) {
            to_delete.add(i)
        }
    }
    // Delete the rows we do not need
    for (i = 0; i < rt.size(); i++) {
        value = to_delete.get(i)
        v = value-i
        rt.deleteRow(v)
    }
    rt.updateResults()
    // Insert values in summary table
    for (i = 0; i < rt.size(); i++) {
        rt.setValue("Dataset", i, dataset_name)
        rt.setValue("Bounding_Box", i, max_bounding_box)
        rt.setValue("Channel Index", i, channel_index)
    }
    headings = rt.getHeadings()
    
    for (j = 0; j < rt.size(); j++) {
        row = new ArrayList()
        for (i = 0; i < headings.length; i++) {
            heading = rt.getColumnHeading(i)
            if (heading.equals("Slice") || heading.equals("Dataset") || heading.equals("Label")) {
                row.add(rt.getStringValue(i, j))
            } else {
                row.add(new Double(rt.getValue(i, j)))
            }
        }
        row.add(image)
        table_rows.add(row)
    }
    
    return headings
}


def create_table_columns(headings) {
    "Create the table headings from the ImageJ results table"
    size = headings.size()
    table_columns = new TableDataColumn[size+1]
    //populate the headings
    for (h = 0; h < size; h++) {
        heading = headings[h]
        if (heading.equals("Slice") || heading.equals("Dataset") || heading.equals("Label")) {
            table_columns[h] = new TableDataColumn(heading, h, String)
        } else {
            table_columns[h] = new TableDataColumn(heading, h, Double)
        }
    }
    table_columns[size] = new TableDataColumn("Image", size, ImageData)
    return table_columns
}

def upload_csv_to_omero(ctx, file, project_id) {
    "Upload the CSV file and attach it to the specified project"
    svc = gateway.getFacility(DataManagerFacility)

    file_size = file.length()
    original_file = new OriginalFileI()
    original_file.setName(rstring(file.getName()))
    original_file.setPath(rstring(file.getAbsolutePath()))
    original_file.setSize(rlong(file_size))
    checksum_algorithm = new ChecksumAlgorithmI()
    checksum_algorithm.setValue(rstring(ChecksumAlgorithmSHA1160.value))
    original_file.setHasher(checksum_algorithm)
    original_file.setMimetype(rstring("text/csv"))
    original_file = svc.saveAndReturnObject(ctx, original_file)
    store = gateway.getRawFileService(ctx)

    // Open file and read stream
    INC = 262144
    pos = 0
    buf = new byte[INC]
    ByteBuffer bbuf = null
    stream = null
    try {
        store.setFileId(original_file.getId().getValue())
        stream = new FileInputStream(file)
        while ((rlen = stream.read(buf)) > 0) {
            store.write(buf, pos, rlen)
            pos += rlen
            bbuf = ByteBuffer.wrap(buf)
            bbuf.limit(rlen)
        }
        original_file = store.save()
    } finally {
        if (stream != null) {
            stream.close()
        }
        store.close()
    }
    // create the file annotation
    namespace = "training.demo"
    fa = new FileAnnotationI()
    fa.setFile(original_file)
    fa.setNs(rstring(namespace))

    data_object = new ProjectData(new ProjectI(project_id, false)) 
    svc.attachAnnotation(ctx, new FileAnnotationData(fa), data_object)
}

def save_summary_as_omero_table(ctx, rows, columns, project_id) {
    "Create an OMERO table with the summary result and attach it to the specified project"
    data = new Object[columns.length][rows.size()]
    for (r = 0; r < rows.size(); r++) {
        row = rows.get(r)
        for (i = 0; i < row.size(); i++) {
            //Due to a limitation of OMERO.parade multiply value by 100
            v = row.get(i)
            if (v instanceof Double) {
                v = v * 100
            }
            data[i][r] = v
        }
    }
    // Create the table
    table_data = new TableData(columns, data)
    table_facility = gateway.getFacility(TablesFacility)
    data_object = new ProjectData(new ProjectI(project_id, false))
    result = table_facility.addTable(ctx, data_object, "Summary_from_Fiji", table_data)
    oid = result.getOriginalFileId()
    // Retrieve the annotation and set the namespace (due to a limitation of JavaGateway)
    annotations = table_facility.getAvailableTables(ctx, data_object)
    it = annotations.iterator()
    while (it.hasNext()) {
        ann = it.next()
        if (ann.getFileID() == oid) {
            ann.setNameSpace(FileAnnotationData.BULK_ANNOTATIONS_NS)
            gateway.getUpdateService(ctx).saveAndReturnObject(ann.asIObject())
            break
        }
    }
}


def save_summary_as_csv(file, rows, columns) {
    "Save the summary locally as a CSV"
    stream = null
    sb = new StringBuilder()
    try {
        stream = new PrintWriter(file)
        l = table_columns.length
        for (i = 0; i < l; i++) {
            sb.append(table_columns[i].getName())
            if (i != (l-1)) {
                sb.append(", ")
            }
        }
        sb.append("\n")
        table_rows.each() { row ->
            size = row.size()
            for (i = 0; i < size; i++) {
                value = row.get(i)
                if (value instanceof ImageData) {
                    sb.append(value.getId())
                } else {
                    sb.append(value)
                }
                if (i != (size-1)) {
                    sb.append(", ")
                }
            }
            sb.append("\n")
        }
        stream.write(sb.toString())
    } finally {
        stream.close()
    }
}

// Main part of the script

exp = gateway.getLoggedInUser()
group_id = exp.getGroupId()

// Load all the images contained in the dataset
println "loading images..."
images = get_images(gateway, dataset_id)

// Load the file annotations linked to the dataset.
println "loading macro..."
macro_file = get_macro_file(gateway, dataset_id)

//Apply the macro to all the images in the dataset
println "analyzing images..."
images.each { img ->
    // if target_user ~= None:
    // Switch context to target user and open omeroImage as ImagePlus object
    img_id = img.getId()
    println "Running macro on image: "+img_id
    imp = open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, String.valueOf(img_id))

    imp = IJ.getImage()
    
    IJ.runMacroFile(macro_file.getAbsolutePath())
    rt = ResultsTable.getResultsTable()
    // Save the ROIs
    roivec = save_rois_to_omero(ctx, id, imp)   
    
    println "creating summary results for image ID " + id
    headings = save_row(rt, table_rows, channel_index, dataset_name, image)
    if (table_columns == null) {
        table_columns = create_table_columns(headings)
    }
    
    imp.changes = false     /// Prevent "Save Changes?" dialog
    imp.close()
}

//Create the result file
tmp_dir = Files.createTempDirectory("Fiji_csv")
path = tmp_dir.resolve("idr0021_merged_results.csv")
file_path = Files.createFile(path)
file = new File(file_path.toString())
// create CSV file
save_summary_as_csv(file, table_rows, table_columns)


upload_csv_to_omero(ctx, file, project_id)
save_summary_as_omero_table(ctx, table_rows, table_columns, project_id)
// delete the local copy of the temporary file and directory
dir = new File(tmp_dir.toString())
entries = dir.listFiles()
for (i = 0; i < entries.length; i++) {
    entries[i].delete()
}
dir.delete()

// Close the connection
gateway.disconnect()
//Delete macrofile
macro_file.delete()
println "processing done"

loading images...
loading macro...
analyzing images...
Running macro on image: 9539
